In [21]:
# Model creation and training.
import torch
from functions import get_causal_mask
from full_transformer import Transformer
from torch.optim import Adam

# Data download and decompression.
import os
import requests
import gzip
import shutil

# Data processing
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
import spacy

In [22]:
# Set device to gpu if available
device = torch.device('cuda' if torch.cuda.is_available() 
                      else 'mps' if torch.backends.mps.is_available() 
                      else 'cpu')
print("You are using device: %s" % device)

You are using device: mps


## Download and decompress the training, validation and test data.

In [23]:
MULTI30_URL = "https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/"
LOCAL_DATA_DIR = "multi30k"
DATA_FILES_CONFIG = {
    "train": {"en": "train.en.gz", "fr": "train.fr.gz"},
    "val": {"en": "val.en.gz", "fr": "val.fr.gz"},
    "test": {"en": "test_2016_flickr.en.gz", "fr": "test_2016_flickr.fr.gz"},
}
DATA_FILES_NAMES = [
    filename
    for split in DATA_FILES_CONFIG.values()
    for filename in split.values()
]

# Download data into a local directory. 
os.makedirs(LOCAL_DATA_DIR, exist_ok=True)

for file_name in DATA_FILES_NAMES:
    
    local_path = os.path.join(LOCAL_DATA_DIR, file_name)
    decompressed_path = os.path.splitext(local_path)[0]
    
    if not os.path.exists(decompressed_path):
        
        # Download file.
        with requests.get(os.path.join(MULTI30_URL, file_name), stream=True) as response:
            response.raise_for_status()
            with open(local_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

        # Decompress file.
        with gzip.open(local_path, 'rb') as f_in:
            with open(decompressed_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)    

        # Remove compressed version of file.
        os.remove(local_path)

## Load the data from files into ram, and tokenize.

In [24]:
# To save time, save the data object. That way, spacy only needs tokenize the data once.
if os.path.exists("data_cache.pt"):
    data = torch.load("data_cache.pt")
else:
    spacy_en = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
    spacy_fr = spacy.load("fr_core_news_sm", disable=['parser', 'ner'])

    def load_data(eng_file_path, fr_file_path):
        
        pairs = []
        with open(eng_file_path, 'r') as f1, open(fr_file_path, 'r') as f2:
            for eng_line, fr_line in zip(f1, f2):
                eng_tokens = [token.text.lower() for token in spacy_en(eng_line.strip())]
                fr_tokens = [token.text.lower() for token in spacy_fr(fr_line.strip())]
                pairs.append((eng_tokens, fr_tokens))
                
        return pairs

    data = {}  
    for split, langs in DATA_FILES_CONFIG.items():
        eng_file_path = os.path.join(LOCAL_DATA_DIR, os.path.splitext(langs['en'])[0])
        fr_file_path = os.path.join(LOCAL_DATA_DIR, os.path.splitext(langs['fr'])[0])
        
        data[split] = load_data(eng_file_path, fr_file_path)
    
    torch.save(data, "data_cache.pt")
    
data['train'][0] # Example

/var/folders/fh/1y5zmgwd69b0g1w1mbx7cng00000gn/T/ipykernel_63930/1907428772.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("data_cache.pt")


(['two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.'],
 ['deux',
  'jeunes',
  'hommes',
  'blancs',
  'sont',
  'dehors',
  'près',
  'de',
  'buissons',
  '.'])

## Create English and French vocabularies.

In [25]:
PAD_TOKEN, PAD_IDX = "<PAD>", 0
SOS_TOKEN, SOS_IDX = "<SOS>", 1
EOS_TOKEN, EOS_IDX = "<EOS>", 2
UNK_TOKEN, UNK_IDX = "<UNK>", 3

class Vocab:
    
    def __init__(self):
        
        self.word_to_ordinal = {PAD_TOKEN: PAD_IDX, SOS_TOKEN: SOS_IDX, EOS_TOKEN: EOS_IDX, UNK_TOKEN: UNK_IDX}
        self.ordinal_to_word = {PAD_IDX: PAD_TOKEN, SOS_IDX: SOS_TOKEN, EOS_IDX: EOS_TOKEN, UNK_IDX: UNK_TOKEN}
        self.count = 4
    
    # This method builds the vocabulary, for each sentence passed in.
    def add_sentence(self, sentence):
        
        for word in sentence:
            if word not in self.word_to_ordinal:
                self.word_to_ordinal[word] = self.count
                self.ordinal_to_word[self.count] = word
                self.count += 1
    
    # This method is for creating model inputs.
    def to_ordinals(self, sentence):
        
        ordinal_sentence = [SOS_IDX]
        for word in sentence:
            ordinal_sentence.append(self.word_to_ordinal.get(word, UNK_IDX))
        ordinal_sentence.append(EOS_IDX)
            
        return torch.tensor(ordinal_sentence, dtype=torch.int64)
    
    # This method is for viewing model outputs.
    def to_words(self, ordinal_sentence):
        
        tokens = []
        for ordinal in ordinal_sentence:
            ordinal = ordinal.item()
            if ordinal == EOS_IDX:
                break
            if ordinal != SOS_IDX and ordinal != PAD_IDX:
                tokens.append(self.ordinal_to_word.get(ordinal, UNK_TOKEN))
        
        return ' '.join(tokens)
    
# Build the vocabularies from training data.
en_vocab = Vocab()
fr_vocab = Vocab()

for en_fr_pair in data['train']:
    en_vocab.add_sentence(en_fr_pair[0])
    fr_vocab.add_sentence(en_fr_pair[1])

## Creating a dataloader for train, val, and test datasets.

In [26]:
class Multi30k(Dataset):
    
    def __init__(self, pairs, en_vocab, fr_vocab):
        super().__init__()
        
        self.ordinal_pairs = []
        for pair in pairs:
            ordinal_pair = (en_vocab.to_ordinals(pair[0]), fr_vocab.to_ordinals(pair[1]))
            self.ordinal_pairs.append(ordinal_pair)
            
    def __len__(self):
        return len(self.ordinal_pairs)
        
    def __getitem__(self, index):
        return self.ordinal_pairs[index]

In [27]:
BATCH_SIZE = 16

def collate_fn(batch):
    X_src = pad_sequence([batch[i][0] for i in range(len(batch))], batch_first=True, padding_value=PAD_IDX)
    X_tgt = pad_sequence([batch[i][1] for i in range(len(batch))], batch_first=True, padding_value=PAD_IDX)
    return (X_src, X_tgt)

dataloaders = {}
for split, pairs in data.items():
    dataset = Multi30k(pairs, en_vocab, fr_vocab)
    dataloaders[split] = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=(split == "train"),  # Shuffle only True for training data
        collate_fn=collate_fn,
    )

## Model creation and training loop.

In [28]:
CONTEXT_SIZE = 60 # The number of positional encodings to learn.

encoder_config = {
    "vocab_size": en_vocab.count,
    "context_size": 60,
    "stack_size": 6,
    "num_heads": 8,
    "hidden_size": 512,
    "key_size": 64,
    "value_size": 64,
    "feedforward_size": 2048,
    "dropout": 0.1,
}

decoder_config = {
    "vocab_size": fr_vocab.count,
    "context_size": 60,
    "stack_size": 6,
    "num_heads": 8,
    "hidden_size": 512,
    "key_size": 64,
    "value_size": 64,
    "feedforward_size": 2048,
    "dropout": 0.1,
}

model = Transformer(encoder_config, decoder_config).to(device)
criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = Adam(model.parameters(), lr=0.0001)

epochs_completed = 0

if os.path.exists("checkpoint.pt"):
    checkpoint = torch.load("checkpoint.pt")
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    epochs_completed = checkpoint["epochs_completed"]

model.train()

epochs = 5
for _ in range(epochs):
    for batch_src, batch_tgt in dataloaders["train"]:

        # Move batch to gpu, prepare model inputs.
        encoder_in = batch_src.to(device)
        decoder_in = batch_tgt[:, :-1].to(device)  # Do not include the last token.
        ground_truth = batch_tgt[:, 1:].to(
            device
        )  # Do not include the first token. The ground truth for the SOS token is thus the first word of the French sentence.

        # Create masks.
        tgt_len = decoder_in.shape[1]

        tgt_causal_mask = get_causal_mask(tgt_len, device)
        tgt_key_padding_mask = decoder_in == PAD_IDX
        src_key_padding_mask = encoder_in == PAD_IDX

        # Update weights
        optimizer.zero_grad()

        features = model(
            decoder_in,
            encoder_in,
            tgt_causal_mask,
            tgt_key_padding_mask,
            src_key_padding_mask,
        )
        loss = criterion(features.view(-1, features.shape[-1]), ground_truth.view(-1))

        print(loss)

        loss.backward()
        optimizer.step()

    epochs_completed += 1

torch.save(
    {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "epochs_completed": epochs_completed,
    },
    "checkpoint.pt",
)

/var/folders/fh/1y5zmgwd69b0g1w1mbx7cng00000gn/T/ipykernel_63930/538358931.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("checkpoint.pt")


tensor(0.5539, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.7620, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.0448, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.7477, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.6721, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.6577, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.5694, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.6373, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.8127, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.6860, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.7937, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.6750, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.7131, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.5208, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.6838, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.6679, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.8753, device='mps:0', grad_fn=<NllLossBackward0

In [32]:
# The number of epochs completed in this demo notebook, for the example below:
epochs_completed

15

In [29]:
batches = iter(dataloaders['train'])

X_src, X_tgt = next(batches)
X_src = X_src.to(device)
X_tgt = X_tgt.to(device)

X_src = X_src[:1,:]
src_key_padding_mask = (X_src == PAD_IDX)

model.eval()
with torch.no_grad():
    sentence = model.generate(X_src, src_key_padding_mask, 7, 60, SOS_IDX, PAD_IDX)
    
print("English sentence:", en_vocab.to_words(X_src[0]))
print("French sentence ground truth:", fr_vocab.to_words(X_tgt[0]))
print("Model output:", fr_vocab.to_words(sentence))

English sentence: a man is preparing food on a grill outside a home .
French sentence ground truth: un homme prépare de la nourriture sur un barbecue devant une maison .
Model output: un homme prépare de la nourriture sur un barbecue dehors .
